In [1]:
from multiprocessing import Pool
import numpy as np
import pyvinecopulib as pv

from vcimpute.helper_datagen import make_complete_data_matrix, mask_MCAR
from gcimpute.helper_evaluation import get_smae
from itertools import permutations

In [2]:
d = 10
X = make_complete_data_matrix(1000, d, 'gaussian', seed=10)
X_mask = mask_MCAR(X, 'general', .2, seed=10)

In [4]:
from vcimpute.helper_mdp import all_mdps

In [13]:
X_mask.shape

(1000, 10)

In [45]:
X_mis = X_mask

In [53]:
def all_mdps_idx(X_mis):
    a = np.zeros(np.prod(X_mis.shape), dtype=np.uint64)
    b = np.flatnonzero(np.isnan(X_mis))
    a[b] = 1 + b % X_mis.shape[1]
    a = a.reshape(X_mis.shape)
    return a

In [68]:
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian])

In [71]:
cop = pv.Vinecop(d=d)

In [72]:
cop.select(X_mask, controls=controls)

In [76]:
T = cop.matrix

In [77]:
def adhoc_patterns(T):
    pass

In [ ]:
make_diago

In [86]:
T2 = np.zeros(shape=(d, d), dtype=np.uint64)
for j in range(d-1):
    for i in range(d-j-1):
        T2[i,j] = cop.structure.struct_array(i, j, natural_order=True)
for j in range(d):
    T2[d-j-1,j] = 1++j
    

In [185]:
structure = pv.RVineStructure(d=5)

In [186]:
T = pv.Vinecop(structure).matrix

In [234]:
T

array([[2, 3, 4, 5, 5],
       [3, 4, 5, 4, 0],
       [4, 5, 3, 0, 0],
       [5, 2, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint64)

In [188]:
d = T.shape[1]

In [223]:
def is_diagonal(T):
    d = T.shape[1]
    diagonal = True
    for j in range(d-1):
        diagonal &= T[d-j-2, j] == T[d-j-2, j+1]
    return diagonal

In [245]:
ced, cing, _ = get_ced_cing(T)
T2 = np.zeros(shape=(d,d),dtype=np.uint64)

j=0
T2[d-j-1,0] = T[d-j-1,0]
T2[d-j-2,0] = T[d-j-2,1]

for k in range(d-j-2):
    assert T2[k,j]==0
    T2[k, j] = find(T2[d-j-1,j], k, ced, cing)


for j in range(1,d-1):
    if T2[d-j-1,j] != T2[d-j-1,j-1]:
        T2[d-j-1,j] = T2[d-j-1,j-1]
        T2[d-j-2,j] = T[d-j-1,j]
    
    for k in range(d-j-2):
        assert T2[k,j]==0
        T2[k, j] = find(T2[d-j-1,j], k, ced, cing)
T2[0,d-1] = T2[0,d-2]

In [248]:
T2

array([[2, 1, 1, 4, 4],
       [3, 4, 3, 3, 0],
       [4, 2, 2, 0, 0],
       [2, 2, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint64)

In [239]:
pv.Vinecop(T)

<pyvinecopulib.Vinecop>
** Tree: 0
1,2 <-> Independence
2,3 <-> Independence
3,4 <-> Independence
4,5 <-> Independence
** Tree: 1
1,3 | 2 <-> Independence
2,4 | 3 <-> Independence
3,5 | 4 <-> Independence
** Tree: 2
1,4 | 3,2 <-> Independence
2,5 | 4,3 <-> Independence
** Tree: 3
1,5 | 4,3,2 <-> Independence

In [236]:
T2

array([[2, 4, 1, 4, 4],
       [3, 3, 3, 3, 0],
       [4, 2, 2, 0, 0],
       [5, 5, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint64)

In [231]:
T2

array([[2, 4, 1, 4, 4],
       [3, 3, 3, 3, 0],
       [4, 2, 2, 0, 0],
       [5, 5, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint64)

In [227]:
pv.Vinecop(stru)

RuntimeError: not a valid R-vine array: the antidiagonal entry of a column must not be contained in any column further to the right.

In [233]:
ced

[[1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [2, 3],
 [2, 4],
 [2, 5],
 [3, 4],
 [3, 5],
 [4, 5]]

In [232]:
cing

[[], [2], [2, 3], [2, 3, 4], [], [3], [3, 4], [], [4], []]

In [229]:
find(2,0, ced,cing)

1

In [194]:
ced

[[1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [2, 3],
 [2, 4],
 [2, 5],
 [3, 4],
 [3, 5],
 [4, 5]]

In [198]:
T2

array([[2, 0, 0, 4, 4],
       [3, 0, 3, 3, 0],
       [4, 2, 2, 0, 0],
       [5, 5, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint64)

4

In [124]:
pv.Vinecop(structure)

<pyvinecopulib.Vinecop>
** Tree: 0
1,2 <-> Independence
2,3 <-> Independence
3,4 <-> Independence
** Tree: 1
1,3 | 2 <-> Independence
2,4 | 3 <-> Independence
** Tree: 2
1,4 | 3,2 <-> Independence

In [126]:
pv.Vinecop(pv.RVineStructure(np.array([
[2, 3, 3, 3],
[3, 2, 2, 0],
[4, 4, 0, 0],
[1, 0, 0, 0]
], dtype=np.uint64)))

<pyvinecopulib.Vinecop>
** Tree: 0
1,2 <-> Independence
4,3 <-> Independence
2,3 <-> Independence
** Tree: 1
1,3 | 2 <-> Independence
4,2 | 3 <-> Independence
** Tree: 2
1,4 | 3,2 <-> Independence

In [104]:
from vcimpute.helper_diagonalize import get_ced_cing, find

In [111]:
ced, cing, _ = get_ced_cing(T)

In [95]:
T2[:-1,1:]

array([[ 4,  4,  9,  6,  9,  9, 10, 10, 10],
       [ 3,  9,  6,  9, 10, 10,  9,  9,  0],
       [ 9,  6,  5, 10,  7,  8,  8,  0,  0],
       [ 6,  5, 10,  7,  8,  7,  0,  0,  0],
       [ 5, 10,  7,  8,  6,  0,  0,  0,  0],
       [10,  7,  8,  5,  0,  0,  0,  0,  0],
       [ 7,  8,  4,  0,  0,  0,  0,  0,  0],
       [ 8,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=uint64)

In [94]:
cpv.RVineStructure(T2[:-1,1:])

RuntimeError: not a valid R-vine array: the order/antidiagonal must contain the numbers 1, ..., d (the number of variables)

In [89]:
pv.RVineStructure(T2)

<pyvinecopulib.RVineStructure>
6 4 4 9 6 9 9 10 10 10 
9 3 9 6 9 10 10 9 9 
4 9 6 5 10 7 8 8 
3 6 5 10 7 8 7 
2 5 10 7 8 6 
5 10 7 8 5 
10 7 8 4 
7 8 3 
8 2 
1 

In [87]:
T2

array([[ 6,  4,  4,  9,  6,  9,  9, 10, 10, 10],
       [ 9,  3,  9,  6,  9, 10, 10,  9,  9,  0],
       [ 4,  9,  6,  5, 10,  7,  8,  8,  0,  0],
       [ 3,  6,  5, 10,  7,  8,  7,  0,  0,  0],
       [ 2,  5, 10,  7,  8,  6,  0,  0,  0,  0],
       [ 5, 10,  7,  8,  5,  0,  0,  0,  0,  0],
       [10,  7,  8,  4,  0,  0,  0,  0,  0,  0],
       [ 7,  8,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=uint64)

In [54]:
all_mdps_idx(X_mask)

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 9, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 8, 0, 0],
       [1, 2, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint64)